In [2]:
## build researcher events, student events, and self-study events
## researcher are simply those with NH citations

In [3]:
import sys
import pandas as pd
import time
import datetime

In [4]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('rfm_data')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


2022-02-20 18:16:00,457| ERROR   | Could not open connection to gateway


[ERROR] [sshtunnel - sshtunnel.SSHTunnelForwarder]: Could not open connection to gateway [sshtunnel._connect_to_gateway:1427]


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

## Figure: active simulation users by geolocation

In [ ]:
## need to do iterative importing once again

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
## caching check 
try:
    user_breakdown_df = pd.read_csv(cwd+'/cached_data/geo_nh_user_breakdown.csv')
    print(user_breakdown_df)
    cache_flag = True
except:
    cache_flag = False

In [ ]:
## determine starting year
if cache_flag == False:
    start_year_base = 1999 #2000
else:
#     print('check the saved file')
    start_year_base = int(user_breakdown_df['year_month'].to_list()[-1][:4])

In [ ]:
start_year_base

In [ ]:
import datetime
today = datetime.datetime.today()

In [ ]:
all_months_list = list()
for this_year in range(start_year_base+1,today.year+2):
    if cache_flag == True:
        for this_month in range(1,today.month+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))
    else:
        for this_month in range(1,12+1):
            all_months_list.append(datetime.datetime(this_year, this_month, 1))

In [ ]:
country_df = pd.DataFrame()
if cache_flag == False:
    if today.month != 12:
        country_df['year_month'] = all_months_list[:-12-(12-today.month)]

In [ ]:
country_df

In [ ]:
usa = []
china = []
eu = []
india = []
other = []
eu_list = ['BE','BG','EL','CZ','DK','DE','EE','IE','ES','FR','HR','IT','CY','LV',\
          'LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE','IS',\
           'NO','LI','CH','UK','ME','MK','AL','RS','TR','BA','XK','AM','AZ','BY',\
           'MD','GE','UA','RU']

In [ ]:
from copy import deepcopy
import numpy as np

In [ ]:
## computer has memory limits, so split toolstart into branches
start_year = deepcopy(start_year_base)

while start_year < today.year+1: #datetime.datetime.now().year:
    start_year += 1

    for i in range(1,13):
        if i < 10:
            start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
        else:
            start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"

        sql_query_country = "select distinct user, countryip from toolstart where user != 'instanton' "\
        + "and user != 'gridstat' and datetime <= "+end_date+" and datetime >= "+start_date
        country_users = pd.read_sql_query(sql_query_country, nanohub_metrics_db)#.drop_duplicates()
        
        # run geography filter
        country_ipds = country_users['countryip'].to_list()
        
        bool_US = np.in1d(np.array(country_ipds),np.array(['US']))
        bool_EU = np.in1d(np.array(country_ipds),np.array(eu_list))
        bool_CN = np.in1d(np.array(country_ipds),np.array(['CN']))
        bool_IN = np.in1d(np.array(country_ipds),np.array(['IN']))
#         bool_OT = bool_US | bool_EU | bool_CN | bool_IN
        
        usa.append(len(np.where(bool_US)[0].tolist()))
        eu.append(len(np.where(bool_EU)[0].tolist()))
        china.append(len(np.where(bool_CN)[0].tolist()))
        india.append(len(np.where(bool_IN)[0].tolist()))
        other.append(len(country_ipds)-usa[-1]-eu[-1]-china[-1]-india[-1]) # len(np.where(bool_OT)[0].tolist()))
        
#         country_users.append(country_users.drop_duplicates().shape[0])

    print(start_year)

In [ ]:
country_df.shape

In [ ]:
len(country_df['year_month'].to_list())

In [ ]:
print(len(usa))
usa2 = usa[12:][:len(country_df['year_month'].to_list())]
print(len(usa2))

In [ ]:
country_df['USA'] = usa[:len(country_df['year_month'].to_list())]
country_df['China'] = china[:len(country_df['year_month'].to_list())]
country_df['Europe'] = eu[:len(country_df['year_month'].to_list())]
country_df['India'] = india[:len(country_df['year_month'].to_list())]
country_df['Other'] = other[:len(country_df['year_month'].to_list())]

In [ ]:
country_df

In [ ]:
country_df['Total'] = \
np.array(usa[:len(country_df['year_month'].to_list())])+\
np.array(china[:len(country_df['year_month'].to_list())])+\
np.array(eu[:len(country_df['year_month'].to_list())])+\
np.array(india[:len(country_df['year_month'].to_list())])+\
np.array(other[:len(country_df['year_month'].to_list())])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import re

In [ ]:
## read the dev_flags.txt to determine where to save
with open(cwd+'/dev_flags.txt','r') as f:
    raw_flags = f.read()
flag_pattern = re.compile(r'= \w+')
flags1 = re.findall(flag_pattern,raw_flags)    
flags_result = [i[2:] for i in flags1]
dev_flag = flags_result[0]
prod_flag = flags_result[1]

In [ ]:
plt.figure(figsize=(9,6))
plt.stackplot(country_df['year_month'], \
              country_df[country_df.columns.to_list()[1:-1]].to_numpy().T, \
             labels=country_df.columns.to_list()[1:-1], \
             colors=['blue', 'red', 'orange', 'green', 'black'],\
             edgecolor='white');

plt.legend(loc='upper left');
plt.ylabel('12-month trailing total')
plt.xlim([datetime.datetime(2002,1,1), datetime.datetime(today.year,today.month,1)])

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months.png', dpi=300, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months.png', dpi=300, bbox_inches='tight')
else:
    print('no printing flag is active')
# plt.show()

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(country_df['year_month'], \
         country_df['USA'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='US', color='blue');
plt.plot(country_df['year_month'], \
         country_df['China'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='China', color='red');
plt.plot(country_df['year_month'], \
         country_df['Europe'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='Europe', color='orange');
plt.plot(country_df['year_month'], \
         country_df['India'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='India', color='green');
plt.plot(country_df['year_month'], \
         country_df['Other'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='Others', color='black');

plt.xlim([datetime.datetime(2002,1,1), datetime.datetime(today.year,today.month,1)])
plt.ylim([0, 100])
plt.legend(loc='upper right');
plt.ylabel('Percentage (%)')

if dev_flag == 'True':
    plt.savefig(cwd+'/plots_local/300_simusers_geo_12months_percent.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/300_simusers_geo_12months_percent.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_local/300_simusers_geo_12months_percent.png', dpi=300, bbox_inches='tight')
elif prod_flag == 'True':
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months_percent.eps', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months_percent.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(cwd+'/plots_production/300_simusers_geo_12months_percent.png', dpi=300, bbox_inches='tight')
else:
    print('no printing flag is active')

# plt.show()

In [ ]:
country_df.to_csv(cwd+'/cached_data/simuser_geo_12months.csv')